In [1]:
# AAVE - AVAX Link
# https://app.aave.com/#/reserve-overview/0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee-0xb31f66aa3c1e785363f0875a1b74e27b85fd66c70xb6a86025f0fe1862b372cb0ca18ce3ede02a318f

In [2]:
import time
import datetime
import math
import pandas as pd
import numpy as np
import xlwings as xw
import os

In [3]:
# Setting
start_time=time.time()

token='AVAX'

general_path=os.path.abspath(os.path.dirname(os.getcwd()))
result_template_path=general_path+'\\Result\\'+str(token)+'\\'+'Result Template.xlsx'

# Initial Token Amount
initial_token_amt = 100

# Info on the AAVE web page
# https://app.aave.com/#/deposit
ltv_threshold=0.65
max_ltv=0.5
ltv_list=[0.5]

deposit_apy=0.0165
reward_apr=0.0546

borrow_stable_apy=None
borrow_variable_apy=0.0562

# in year
time_1=1

# times to deosit and borrow
loop_count=4

# Token Price
token_price=72

ltv=ltv_list[0]

In [4]:
class aave_recursive_borrowing_strategy:
    def __init__(self, DepositAmount:float, TokenPrice:float, LTV:float, DepostiAPY:float, \
                 RewardAPY:float, Time:float, BorrowAPY:float):
        self.DepositAmount=DepositAmount
        self.TokenPrice=TokenPrice
        self.LTV=LTV
        self.DepostiAPY=DepostiAPY
        self.RewardAPY=RewardAPY
        self.Time=Time
        self.BorrowAPY=BorrowAPY
    
    def value(self):
        self.DepositValue = self.DepositAmount * self.TokenPrice
        self.BorrowValue = self.DepositValue * self.LTV
        self.BorrowAmount = self.BorrowValue / self.TokenPrice
        
        return self.DepositValue, self.BorrowValue, self.BorrowAmount
    
    def income(self):
        self.DepositIncome = self.DepositAmount * self.TokenPrice * self.DepostiAPY * self.Time
        self.RewardIncome = self.DepositAmount * self.TokenPrice * self.RewardAPY * self.Time
        self.TotalIncome = self.DepositIncome + self.RewardIncome
        
        return self.DepositIncome, self.RewardIncome, self.TotalIncome
    
    # Interest Rate
    def cost(self):
        self.BorrowCost = self.BorrowAmount * self.TokenPrice * self.BorrowAPY * self.Time
        
        return self.BorrowCost
    
    def net_income(self):
        self.NetIncome = self.TotalIncome - self.BorrowCost
        self.Net_Income_Token_Amount = self.NetIncome/self.TokenPrice
        
        return self.NetIncome, self.Net_Income_Token_Amount
    
    def apy(self):
        self.Dep_Borr_APY=self.NetIncome/self.DepositValue/self.Time
        
        return self.Dep_Borr_APY
    


In [5]:
# Initializing
borrow_amt=initial_token_amt
sum_net_income_token_amt=0
sum_borrow_amt=0

col_header_list=['Deposit Amount', 'Borrow Amount', 'Net Income (/Token)', 'APY']
dep_borr_record_pd = pd.DataFrame(None, index=list(range(1,loop_count+1)), columns=col_header_list)


In [6]:

for i in range(loop_count):
    deposit_amt=borrow_amt
    strategy=aave_recursive_borrowing_strategy(DepositAmount=deposit_amt, TokenPrice=token_price, LTV =ltv, DepostiAPY=deposit_apy, \
                     RewardAPY=reward_apr, Time=time_1, BorrowAPY=borrow_variable_apy)

    value=strategy.value()
    income=strategy.income()
    cost=strategy.cost()
    net_income=strategy.net_income()
    apy=strategy.apy()
    
    # TODO: change borrow_amt to AAVE client txn receipt output
    borrow_amt=value[2]
    net_income_in_token=net_income[1]

    sum_borrow_amt=sum_borrow_amt+borrow_amt
    sum_net_income_token_amt=sum_net_income_token_amt+net_income[1]
    
    # Result for each time, write to DataFrame
    data_list=[deposit_amt, borrow_amt, net_income_in_token, apy]
    dep_borr_record_pd.iloc[i]=data_list

collateral_ratio=sum_borrow_amt/initial_token_amt
apy_after_loop=sum_net_income_token_amt/initial_token_amt/time_1
print()
print('Total Borrow Amount =',sum_borrow_amt)
print('Total Net Income (/Token) =',sum_net_income_token_amt)

print('Collateral Ratio =', collateral_ratio)
print('APY after Loop = ',apy_after_loop)

print()
print('Result for Each Time')
print(dep_borr_record_pd)


Total Borrow Amount = 93.75
Total Net Income (/Token) = 8.062500000000002
Collateral Ratio = 0.9375
APY after Loop =  0.08062500000000002

Result for Each Time
  Deposit Amount Borrow Amount Net Income (/Token)    APY
1            100          50.0                 4.3  0.043
2           50.0          25.0                2.15  0.043
3           25.0          12.5               1.075  0.043
4           12.5          6.25              0.5375  0.043
